# First, extract the frames from the video
Frame extraction code is based on StackOverflow:
https://stackoverflow.com/questions/33311153/python-extracting-and-saving-video-frames

In [43]:
import cv2
import os

names = ["victor", "rik", "tariq"]

others = 6
othernames = ["other" + str(i) for i in range(others)]

base_folder = "shoe_pics/"

otherFolders = [base_folder + "extracted/others/", base_folder + "cropped/others/"]
for f in otherFolders:
    if not os.path.exists(f):
        os.makedirs(f)

for name in names + othernames:
    extractedFolder = base_folder + "extracted/" + name + "/"
    croppedFolder = base_folder + "cropped/" + name + "/"

    if not os.path.exists(extractedFolder):
        os.makedirs(extractedFolder)

    if not os.path.exists(croppedFolder):
        os.makedirs(croppedFolder)


    vidcap = cv2.VideoCapture(name + '.mp4')
    cont, image = vidcap.read()
    count = 0
    while cont:
        cv2.imwrite(extractedFolder + "frame%d.jpg" % count, image)     # save frame as JPEG file      
        cont, image = vidcap.read()
        count += 1
    print("Finished extracting frames of " + name + "'s shoes!")

Finished extracting frames of victor's shoes!
Finished extracting frames of rik's shoes!
Finished extracting frames of tariq's shoes!
Finished extracting frames of other0's shoes!
Finished extracting frames of other1's shoes!
Finished extracting frames of other2's shoes!
Finished extracting frames of other3's shoes!
Finished extracting frames of other4's shoes!
Finished extracting frames of other5's shoes!


# Setup model

In [44]:
# %tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

2.10.0


In [45]:
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

# Importing libraries

In [46]:
import os
import cv2
import numpy as np
# import tensorflow as tf
import tensorflow.compat.v1 as tf
import sys
# from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

# Import utilites
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Helper functions

In [47]:
def findRange(box):
    # Coords in {TopDist, LeftDist, BotDist, RightDist}
    # Returns a value between -1 and 1 for left and right
    leftDist = box['box'][1]
    rightDist = box['box'][3]
    avg = (rightDist - leftDist) / 2 + (leftDist)
    return avg * 2 - 1
    
def reduceBoxes(boxes, scores, classes):
    '''Function to reduce all the boxes to only the ones being displayed.
    Slightly modified version of Shreyas Vedpathak's version from stackoverflow:
    https://stackoverflow.com/a/68376832/19442761'''
    
    max_boxes_to_draw = boxes.shape[0]
    min_score_thresh=.5
    
    # Loop over all objects
    coordinates = []
    for i in range(min(max_boxes_to_draw, boxes.shape[0])):
        if scores[0][i] > min_score_thresh:
            class_id = int(classes[0][i] + 1)
            coordinates.append({
                "box": boxes[i],
                "score": scores[0][i]
            })

    return coordinates

def cropImg(image, box):
    # box in {TopDist, LeftDist, BotDist, RightDist}, in ratios
    # left, upper, right, and lower
    PIL_image = Image.fromarray(np.uint8(image)).convert('RGB')
    siz = PIL_image.size
    hwhw = np.tile(siz, 2).flatten()

    # Cropping as left, upper, right, and lower
    newbox = np.array([box[1], box[0], box[3], box[2]])
    imgcoords = hwhw * newbox
 
    return PIL_image.crop(imgcoords)

# Setting up and loading the model

In [48]:
%matplotlib inline

# Grab path to current working directory
CWD_PATH = os.getcwd()

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = 'weights/frozen_inference_graph.pb'

# Path to label map file
PATH_TO_LABELS = 'weights/label_map.pbtxt'

# Number of classes the object detector can identify
NUM_CLASSES = 1

# Load the label map.
# Label maps map indices to category names, so that when our convolution
# network predicts `5`, we know that this corresponds to `king`.
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')


# Running the actual classification and extracting the shoe pictures

In [49]:
# Load image using OpenCV and
# expand image dimensions to have shape: [1, None, None, 3]
# i.e. a single-column array, where each item in the column has the pixel RGB value
for name in names + othernames:
    # Define folders to read from and write to
    PATH_TO_IMAGE_FOLDER = base_folder + "extracted/" + name + "/"
    croppedFolder = base_folder + "cropped/" + name + "/"
    
    # Keep track of the amount of extracted shoes
    counter = 0
    
    for filename in os.listdir(PATH_TO_IMAGE_FOLDER):
        if filename.endswith(".jpg"):
            image_read = cv2.imread(os.path.join(PATH_TO_IMAGE_FOLDER, filename))
            image = cv2.cvtColor(image_read, cv2.COLOR_BGR2RGB)
            image_expanded = np.expand_dims(image, axis=0)

            # Perform the actual detection by running the model with the image as input
            (boxes, scores, classes, num) = sess.run([detection_boxes, detection_scores, detection_classes, num_detections],feed_dict={image_tensor: image_expanded})


            coords = reduceBoxes(boxes[0], scores, classes)
            ranges = [findRange(coords[i]) for i in range(len(coords))]
            # Draw the results of the detection (aka 'visulaize the results')
            vis_util.visualize_boxes_and_labels_on_image_array(
                image,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=5,
                min_score_thresh=0.50)

            # All the results have been drawn on image. Now display the image.
            image_og = cv2.cvtColor(image_read, cv2.COLOR_BGR2RGB)

            for box in coords:
                cropped = cropImg(image_og, box['box'])
                cropped.save(croppedFolder + "cropped%d.jpg" % counter)
                counter += 1
    print("Finished cropping " + name + "'s shoes!")

Finished cropping victor's shoes!
Finished cropping rik's shoes!
Finished cropping tariq's shoes!
Finished cropping other0's shoes!
Finished cropping other1's shoes!
Finished cropping other2's shoes!
Finished cropping other3's shoes!
Finished cropping other4's shoes!
Finished cropping other5's shoes!


# Merge all the 'other' folders

In [50]:
import shutil
import re

indices = []

for otherX in othernames:
    croppedSrcFolder = base_folder + "cropped/" + otherX + "/"
    croppedDstFolder = otherFolders[1]
    
    l = len(os.listdir(croppedSrcFolder))
    
    for file in os.listdir(croppedSrcFolder):
        num = int(re.findall(r'\d+', file)[0])
        newName = re.sub(r'\d+', str(num + sum(indices)), file)
        shutil.move(croppedSrcFolder + file, croppedDstFolder + newName)
    
    indices.append(l)
    
    shutil.rmtree(croppedSrcFolder)

# Delete the video frames

In [51]:
shutil.rmtree(base_folder + "extracted/")

# Split data into training and test data

In [52]:
import splitfolders
cropped_folder = base_folder + "cropped/"

splitfolders.ratio(cropped_folder, output=base_folder + "split/", ratio=(0.8, 0.1, 0.1)) 
print("Finished!")

Copying files: 7138 files [00:00, 9373.13 files/s]

Finished!


# Retrain the imagenet network to add our shoes as classes
Based on the transfer learning done in the lab

In [53]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import os.path

MODEL_FILE = "shoes.hd5"

def create_model(num_hidden, num_classes):
    base_model = InceptionV3(include_top = False, weights = "imagenet")
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_hidden, activation="relu")(x)
    predictions = Dense(num_classes, activation="softmax")(x)

    for layer in base_model.layers:
        layer.trainable = False

    model = Model(inputs=base_model.input, outputs=predictions)

    return model

def load_existing(model_file):
    model = load_model(model_file)

    numlayers = len(model.layers)

    for layer in model.layers[:numlayers-3]:
        layer.trainable = False

    for layer in model.layers[numlayers-3:]:
        layer.trainable = True

    return model

def train(model_file, train_path, validation_path, num_hidden=200, num_classes=5, steps=32, num_epochs=20):
    if os.path.exists(model_file):
        print("\nExisting model found at %s. Loading.\n\n" % model_file)
        model = load_existing(model_file)
    else:
        print("\nCreating new model.\n\n")
        model = create_model(num_hidden, num_classes)

    model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
    checkpoint = ModelCheckpoint(model_file)
    train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip = True)
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(train_path, target_size=(249,249), batch_size=32, class_mode="categorical")
    validation_generator = test_datagen.flow_from_directory(validation_path, target_size=(249,249), batch_size=32, class_mode="categorical")
    model.fit_generator(train_generator, steps_per_epoch = steps, epochs = num_epochs, callbacks = [checkpoint], validation_data=validation_generator, validation_steps=50)

    for layer in model.layers[:249]:
        layer.trainable = False

    for layer in model.layers[249:]:
        layer.trainable = True

    model.compile(optimizer=SGD(lr=0.00001, momentum=0.9), loss="categorical_crossentropy")

    model.fit_generator(train_generator, steps_per_epoch=steps, epochs = num_epochs, callbacks = [checkpoint], validation_data=validation_generator, validation_steps=50)

# Train the model
train_path = base_folder + "split/train/"
val_path = base_folder + "split/val/"
amt_classes = sum(os.path.isdir(train_path + f) for f in os.listdir(train_path))

train(MODEL_FILE, train_path=train_path, validation_path=val_path, num_classes=amt_classes, steps=110, num_epochs=10)


Creating new model.


Found 5708 images belonging to 4 classes.
Found 712 images belonging to 4 classes.


/tmp/ipykernel_32645/3691090310.py:54: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch = steps, epochs = num_epochs, callbacks = [checkpoint], validation_data=validation_generator, validation_steps=50)


Epoch 1/10
110/110 [==============================] - ETA: 0s - loss: 0.4423WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 50 batches). You may need to use the repeat() function when building your dataset.


INFO:tensorflow:Assets written to: shoes.hd5/assets


INFO:tensorflow:Assets written to: shoes.hd5/assets


...

INFO:tensorflow:Assets written to: shoes.hd5/assets


110/110 [==============================] - 257s 2s/step - loss: 0.0403
Epoch 10/10
110/110 [==============================] - ETA: 0s - loss: 0.0371

INFO:tensorflow:Assets written to: shoes.hd5/assets


INFO:tensorflow:Assets written to: shoes.hd5/assets


110/110 [==============================] - 256s 2s/step - loss: 0.0371


In [66]:
from tensorflow.python.keras.backend import set_session

session = tf.compat.v1.Session(graph = tf.compat.v1.Graph())

samplePath = base_folder + "split/test/"

catgs = {0:"others", 1:"rik", 2:"tariq", 3:"victor"}

def classify(model, image):
    with session.graph.as_default():
        set_session(session)
        result = model.predict(image)
        themax = np.argmax(result)

    return (catgs[themax], result[0][themax], themax)

def load_image(image_fname):
    img = Image.open(image_fname)
    img = img.resize((249, 249))
    imgarray = np.array(img)/255.0
    final = np.expand_dims(imgarray, axis=0)
    return final

score = {"Correct": 0, "Incorrect": 0}

with session.graph.as_default():
    set_session(session)
    model = load_model(MODEL_FILE)

    for catg in names + ["others"]:
        path = samplePath + catg + "/"
        sample_files = os.listdir(path)

        for filename in sample_files:
            filename = os.path.join(path, filename)
            img = load_image(filename)
            label,prob,_ = classify(model, img)

            print("Predict that %s is %s with certainty %s" % (filename, label, prob))
            if label != catg:
                score["Incorrect"] += 1
            else:
                score["Correct"] += 1
print("Accuracy: %s" % str(score["Correct"] / (score["Correct"] + score["Incorrect"])))

/home/rik/.local/lib/python3.8/site-packages/keras/engine/training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Predict that shoe_pics/split/test/victor/cropped624.jpg is victor with certainty 0.99618584
Predict that shoe_pics/split/test/victor/cropped604.jpg is victor with certainty 0.9995191
Predict that shoe_pics/split/test/victor/cropped250.jpg is victor with certainty 0.991886
Predict that shoe_pics/split/test/victor/cropped342.jpg is victor with certainty 0.9993205
Predict that shoe_pics/split/test/victor/cropped148.jpg is victor with certainty 0.9996438
Predict that shoe_pics/split/test/victor/cropped16.jpg is others with certainty 0.8486336
Predict that shoe_pics/split/test/victor/cropped67.jpg is victor with certainty 0.99993837
Predict that shoe_pics/split/test/victor/cropped404.jpg is victor with certainty 0.9999534
Predict that shoe_pics/split/test/victor/cropped654.jpg is victor with certainty 0.9993993
Predict that shoe_pics/split/test/victor/cropped656.jpg is victor with certainty 0.999783
Predict that shoe_pics/split/test/victor/cropped28.jpg is victor with certainty 0.99909616
P

Predict that shoe_pics/split/test/rik/cropped1302.jpg is rik with certainty 0.99986684
Predict that shoe_pics/split/test/rik/cropped845.jpg is rik with certainty 0.99991417
Predict that shoe_pics/split/test/rik/cropped39.jpg is rik with certainty 0.99981076
Predict that shoe_pics/split/test/rik/cropped331.jpg is rik with certainty 0.9994754
Predict that shoe_pics/split/test/rik/cropped583.jpg is rik with certainty 0.9995926
Predict that shoe_pics/split/test/rik/cropped456.jpg is rik with certainty 0.98921704
Predict that shoe_pics/split/test/rik/cropped142.jpg is rik with certainty 0.9983271
Predict that shoe_pics/split/test/rik/cropped857.jpg is rik with certainty 0.9994492
Predict that shoe_pics/split/test/rik/cropped681.jpg is rik with certainty 0.99994767
Predict that shoe_pics/split/test/rik/cropped399.jpg is rik with certainty 0.99975604
Predict that shoe_pics/split/test/rik/cropped4.jpg is rik with certainty 0.99998224
Predict that shoe_pics/split/test/rik/cropped991.jpg is rik 

Predict that shoe_pics/split/test/rik/cropped613.jpg is rik with certainty 0.9947758
Predict that shoe_pics/split/test/rik/cropped107.jpg is rik with certainty 0.9519898
Predict that shoe_pics/split/test/rik/cropped983.jpg is rik with certainty 0.99868196
Predict that shoe_pics/split/test/rik/cropped1096.jpg is rik with certainty 0.9999734
Predict that shoe_pics/split/test/rik/cropped341.jpg is rik with certainty 0.99912256
Predict that shoe_pics/split/test/rik/cropped157.jpg is rik with certainty 0.99918944
Predict that shoe_pics/split/test/rik/cropped769.jpg is rik with certainty 0.99966705
Predict that shoe_pics/split/test/rik/cropped825.jpg is rik with certainty 0.999985
Predict that shoe_pics/split/test/rik/cropped724.jpg is rik with certainty 0.9982014
Predict that shoe_pics/split/test/rik/cropped50.jpg is rik with certainty 0.99311876
Predict that shoe_pics/split/test/rik/cropped762.jpg is rik with certainty 0.99854136
Predict that shoe_pics/split/test/rik/cropped864.jpg is rik 

Predict that shoe_pics/split/test/tariq/cropped898.jpg is tariq with certainty 0.99998116
Predict that shoe_pics/split/test/tariq/cropped65.jpg is tariq with certainty 0.99999964
Predict that shoe_pics/split/test/tariq/cropped792.jpg is tariq with certainty 0.99997485
Predict that shoe_pics/split/test/tariq/cropped780.jpg is tariq with certainty 0.9962465
Predict that shoe_pics/split/test/tariq/cropped636.jpg is tariq with certainty 0.99999905
Predict that shoe_pics/split/test/tariq/cropped12.jpg is tariq with certainty 0.99998593
Predict that shoe_pics/split/test/tariq/cropped984.jpg is tariq with certainty 0.99986637
Predict that shoe_pics/split/test/tariq/cropped570.jpg is tariq with certainty 0.9999993
Predict that shoe_pics/split/test/tariq/cropped667.jpg is tariq with certainty 0.9999999
Predict that shoe_pics/split/test/tariq/cropped806.jpg is tariq with certainty 0.99999785
Predict that shoe_pics/split/test/tariq/cropped690.jpg is tariq with certainty 0.9988951
Predict that sho

Predict that shoe_pics/split/test/others/cropped2782.jpg is others with certainty 0.99949956
Predict that shoe_pics/split/test/others/cropped2183.jpg is others with certainty 0.9999993
Predict that shoe_pics/split/test/others/cropped2156.jpg is others with certainty 0.9975038
Predict that shoe_pics/split/test/others/cropped3251.jpg is others with certainty 1.0
Predict that shoe_pics/split/test/others/cropped1624.jpg is others with certainty 0.99996483
Predict that shoe_pics/split/test/others/cropped86.jpg is others with certainty 1.0
Predict that shoe_pics/split/test/others/cropped11.jpg is others with certainty 1.0
Predict that shoe_pics/split/test/others/cropped3931.jpg is others with certainty 0.9990891
Predict that shoe_pics/split/test/others/cropped2227.jpg is others with certainty 0.99992454
Predict that shoe_pics/split/test/others/cropped1902.jpg is others with certainty 1.0
Predict that shoe_pics/split/test/others/cropped3279.jpg is others with certainty 0.99937314
Predict that

Predict that shoe_pics/split/test/others/cropped757.jpg is others with certainty 0.9999995
Predict that shoe_pics/split/test/others/cropped1722.jpg is others with certainty 0.99999523
Predict that shoe_pics/split/test/others/cropped2394.jpg is others with certainty 0.9995473
Predict that shoe_pics/split/test/others/cropped1550.jpg is others with certainty 0.9999746
Predict that shoe_pics/split/test/others/cropped33.jpg is others with certainty 0.99999523
Predict that shoe_pics/split/test/others/cropped78.jpg is others with certainty 0.9999013
Predict that shoe_pics/split/test/others/cropped3094.jpg is others with certainty 0.9998808
Predict that shoe_pics/split/test/others/cropped3435.jpg is others with certainty 0.99999857
Predict that shoe_pics/split/test/others/cropped2810.jpg is others with certainty 0.9983979
Predict that shoe_pics/split/test/others/cropped1375.jpg is others with certainty 1.0
Predict that shoe_pics/split/test/others/cropped3266.jpg is others with certainty 0.9995

Predict that shoe_pics/split/test/others/cropped2566.jpg is others with certainty 0.9999292
Predict that shoe_pics/split/test/others/cropped3712.jpg is others with certainty 0.9960722
Predict that shoe_pics/split/test/others/cropped1243.jpg is others with certainty 0.9999975
Predict that shoe_pics/split/test/others/cropped1714.jpg is others with certainty 0.9998915
Predict that shoe_pics/split/test/others/cropped2857.jpg is others with certainty 0.999964
Predict that shoe_pics/split/test/others/cropped3355.jpg is others with certainty 0.99999964
Predict that shoe_pics/split/test/others/cropped571.jpg is others with certainty 0.945656
Predict that shoe_pics/split/test/others/cropped2683.jpg is others with certainty 0.9991793
Predict that shoe_pics/split/test/others/cropped3493.jpg is others with certainty 1.0
Predict that shoe_pics/split/test/others/cropped3940.jpg is others with certainty 0.99133646
Predict that shoe_pics/split/test/others/cropped226.jpg is others with certainty 0.9999

Predict that shoe_pics/split/test/others/cropped3044.jpg is others with certainty 0.9983713
Predict that shoe_pics/split/test/others/cropped3063.jpg is others with certainty 0.99546224
Predict that shoe_pics/split/test/others/cropped3642.jpg is others with certainty 0.99994195
Predict that shoe_pics/split/test/others/cropped2727.jpg is others with certainty 0.9999893
Predict that shoe_pics/split/test/others/cropped3392.jpg is others with certainty 0.99999976
Predict that shoe_pics/split/test/others/cropped3016.jpg is others with certainty 0.99989545
Predict that shoe_pics/split/test/others/cropped1718.jpg is others with certainty 0.99999905
Predict that shoe_pics/split/test/others/cropped3618.jpg is others with certainty 0.9999999
Predict that shoe_pics/split/test/others/cropped262.jpg is others with certainty 1.0
Predict that shoe_pics/split/test/others/cropped3640.jpg is others with certainty 0.99999714
Predict that shoe_pics/split/test/others/cropped3696.jpg is others with certainty